In [11]:
import sejong_corpus_cleaner

In [61]:
import re
import kss

In [93]:
REGEX_SPECIAL = '[-~!=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》∎○■★+△▴]'

In [88]:
written_paths = sejong_corpus_cleaner.get_data_paths(corpus_types='written')
colloquial_paths = sejong_corpus_cleaner.get_data_paths(corpus_types='colloquial')

In [89]:
written_paths[2]

'/Users/jaehyunlee/sejong_corpus_cleaner/data/raw/written/BA90A083.txt'

In [ ]:
# 문어체
for written in written_paths:
    with open(written, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        p_tags = soup.find_all('p')

        for p in p_tags:
            p_text = p.text
            p_text = re.sub(REGEX_SPECIAL, '', p_text)
            
            
            for sent in kss.split_sentences(p_text):
                print('sentence : ', sent)

In [64]:
# 구어체
for colloquial in colloquial_paths:
    with open(colloquial, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        p_tags = soup.find_all('p')

        for p in p_tags:
            p_text = p.text
            p_text = re.sub(REGEX_SPECIAL, '', p_text)
            print(p_text)
            
            for sent in kss.split_sentences(p_text):
                print('sentence : ', sent)

[<text>
<group>
<text>
<body>
<source/>
<date>1990/01/10</date>
<page>01</page>
<head>만물상</head>
<p>일본 회사와 거래가 있는 우리나라 회사서 업무 협의차 직원 한 명이 동경 출장을 가면 상대방 회사의 중역들이 공항까지 나와 정중하게 영접을 한다. 그러나 두 명 이상의 한국인들이 일본에 가면 일본 회사 쪽에서는 아무도 공항에 나오지 않는다.</p>
<p>단 한 명의 한국인은 능력도 있고 순발력도 있지만 두 명 이상이 되면 서로 자기만을 내세워서 그냥 내버려 두어도 일본 회사가 득을 볼 수 있기 때문이다. 누군가가 만들어낸 말이 분명하지만 개개인으로서는 누구에게도 뒤지지 않는 한국인들의 특성을 잘 비유한 일화다.</p>
<p>외국인 직업관광 안내원들은 일본인 단체와 한국인 단체는 같은 유교문화권 사람들 같지 않다고 평한다. 일본인들은 항상 리더가 있어야 무슨 일이 있을 때는 리더와 상의하면 모든 일이 잘 풀리는데 한국인들은 모든 사람이 리더라는 이야기다. 누가 한마디 하면 다른 사람들은 요새 유행어로 "잘났어, 정말"이라는 반응만을 보인다.</p>
<p>아시아나 항공이 10일부터 동경에 취항하게 됨으로써 이제는 국제노선도 복수민항시대를 맞게 됐다. 대한항공이 단독으로 나리타공항에 내리다가 이젠 아시아나와 함께 취항하게 된 것이다. 일본은 이미 3 개 항공사가 한일노선을 날고 있으면서, 표면적으로는 경쟁을 표방하면서 승객유치와 노선배분 등은 오순도순 공영의 묘를 살리고 있다. 과당경쟁을 피하고 정부와 협력하여 항공협정 협상과정에서도 공동전략을 편다.</p>
<p>40여년 동안 같은 동포끼리 남북으로 갈려 서로를 비방하고 서로 잘났다면서 살아 온 습성 때문인지 우리 기업들은 외국에 나가서도 정도 이상의 경쟁을 벌여 왔다. 이 같은 무리한 경쟁으로 국가적 손실과 한국인의 자존심이 손상된 적도 많았음은 물론이다. 모처럼 태극 날개를 단 두 민간 항공기가 일본에 취항하는 것을 계기로 이제는 무모